# 성적 + 가치관 기반 추천시스템

- 성적정보와 가치관정보를 모두 코사인유사도를 기반으로 분석하여 적절한 팀메이트 추천
- 성적정보 : 총점은 비슷하고, 영역별 점수는 달라야함
- 가치관정보 : 영역별 점수가 최대한 유사해야함

In [1]:
import numpy as np
import pandas as pd

# 점수 0-10점으로 랜덤 생성
# 인원 수 160명
user_grade = pd.DataFrame(np.random.randint(1,11,size=(160,4)) , index=range(1,161), columns=['데이터분석','머신러닝','웹','디자인'])
# 각 유저(사용자)별 네 점수 총합
user_grade_sum = user_grade.sum(axis=1)

display(user_grade)
display(user_grade_sum)

,데이터분석,머신러닝,웹,디자인
1,8,5,6,5
2,6,8,4,3
3,8,5,5,5
4,7,9,6,1
5,4,9,9,5
...,...,...,...,...
156,5,7,7,7
157,7,8,2,8
158,7,9,7,8
159,6,3,5,5


1      24
2      21
3      23
4      23
5      27
       ..
156    26
157    25
158    31
159    19
160    28
Length: 160, dtype: int64

In [2]:
# 가치관점수 : 
# 디테일 <-> 속도 : 디테일을 중요시하는지 속도를 중요시하는지
# 과정 <-> 결과 : 과정을 중요시하는지 결과를 중요시하는지
# 창업 <-> 취업 : 창업을 꿈꾸는지 취업을 꿈꾸는지
# 아침 <-> 밤 : 아침형인간인지 밤형 인간인지

# 점수 0-10점으로 랜덤 생성
user_style = pd.DataFrame(np.random.randint(1,101,size=(160,4)) , index=range(1,161), columns=['속도','결과','취업','밤'])
user_style['디테일'] = 100 - user_style['속도']
user_style['과정'] = 100 - user_style['결과']
user_style['창업'] = 100 - user_style['취업']
user_style['아침'] = 100 - user_style['밤']
display(user_style)

,속도,결과,취업,밤,디테일,과정,창업,아침
1,43,81,58,95,57,19,42,5
2,59,91,74,39,41,9,26,61
3,79,31,47,99,21,69,53,1
4,88,28,56,71,12,72,44,29
5,50,65,30,82,50,35,70,18
...,...,...,...,...,...,...,...,...
156,56,71,66,15,44,29,34,85
157,57,22,43,47,43,78,57,53
158,20,55,74,19,80,45,26,81
159,91,35,2,29,9,65,98,71


In [3]:
pd.merge(user_grade,user_style, how="outer", left_index=True, right_index=True)

,데이터분석,머신러닝,웹,디자인,속도,결과,취업,밤,디테일,과정,창업,아침
1,8,5,6,5,43,81,58,95,57,19,42,5
2,6,8,4,3,59,91,74,39,41,9,26,61
3,8,5,5,5,79,31,47,99,21,69,53,1
4,7,9,6,1,88,28,56,71,12,72,44,29
5,4,9,9,5,50,65,30,82,50,35,70,18
...,...,...,...,...,...,...,...,...,...,...,...,...
156,5,7,7,7,56,71,66,15,44,29,34,85
157,7,8,2,8,57,22,43,47,43,78,57,53
158,7,9,7,8,20,55,74,19,80,45,26,81
159,6,3,5,5,91,35,2,29,9,65,98,71


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저와 유저 간의 유사도
user_based_collab = cosine_similarity(user_grade, user_grade)
print(user_based_collab)

[[1.         0.92747686 0.99726244 ... 0.95852253 0.98849522 0.9272746 ]
 [0.92747686 1.         0.93313119 ... 0.95246702 0.87177979 0.85375243]
 [0.99726244 0.93313119 1.         ... 0.95763968 0.98335253 0.91598991]
 ...
 [0.95852253 0.95246702 0.95763968 ... 1.         0.9477582  0.96989113]
 [0.98849522 0.87177979 0.98335253 ... 0.9477582  1.         0.95072788]
 [0.9272746  0.85375243 0.91598991 ... 0.96989113 0.95072788 1.        ]]


In [5]:
# 1번 입장에서 진행, 각 점수에서 1번점수를 빼고 절대값을 씌워 1번 사용자의 총합과 얼마나 차이나는지 결정
# 0.1을 곱한 이유? 그대로 사용하면 총합점수차의 영향력이 너무 커진다. 
grade_subs = np.abs(user_grade_sum - user_grade_sum[1]) 

display(grade_subs)

1      0
2      3
3      1
4      1
5      3
      ..
156    2
157    1
158    7
159    5
160    4
Length: 160, dtype: int64

In [6]:
user_based_collab_style = cosine_similarity(user_style, user_style)
print(user_based_collab_style)

[[1.         0.85362083 0.85291461 ... 0.71709607 0.57163345 0.52188148]
 [0.85362083 1.         0.68119275 ... 0.87091209 0.65378207 0.71323769]
 [0.85291461 0.68119275 1.         ... 0.57154644 0.74982574 0.71985215]
 ...
 [0.71709607 0.87091209 0.57154644 ... 1.         0.60641293 0.76521722]
 [0.57163345 0.65378207 0.74982574 ... 0.60641293 1.         0.86069993]
 [0.52188148 0.71323769 0.71985215 ... 0.76521722 0.86069993 1.        ]]


In [7]:
# 높은 추천도를 가지려면?
# user_based_collab 즉, 영역 유사도가 낮아야함 (각 영역별 수치가 최대한 달라야 하므로)
# sub 즉, 총점수차가 낮아야함. (차이가 적어야 비슷한 실력이므로)
# user_based_collab_style 즉, 가치관 유사도는 높아야함

evaluation_value = user_based_collab[0] + grade_subs - user_based_collab_style[0]

# 해당 값을 최종 지표로 사용한다.
evaluation_value

1      2.220446e-16
2      3.073856e+00
3      1.144348e+00
4      1.109179e+00
5      2.943185e+00
           ...     
156    2.214259e+00
157    1.153739e+00
158    7.241426e+00
159    5.416862e+00
160    4.405393e+00
Length: 160, dtype: float64

In [8]:
# 해당 지표가 낮은 순서대로 index 뽑아옴

top10 = evaluation_value.sort_values()[:10].index.tolist()

# 추천되는 인원의 인덱스 리스트
top10

[104, 87, 46, 1, 121, 66, 61, 150, 27, 110]

In [9]:
# 주어진 유저가 어떤 스타일인지 확인

def checkStyle(user):
    style = []
    if user['디테일'] > user['속도']:
        style.append('디테일')
    else:
        style.append('속도')
    if user['과정'] > user['결과']:
        style.append('과정')
    else:
        style.append('결과')
    if user['창업'] > user['취업']:
        style.append('창업')
    else:
        style.append('취업')
    if user['아침'] > user['밤']:
        style.append('아침')
    else:
        style.append('밤')
    return style

In [10]:
# 같은 원리로 인풋 값에 대해 추천시스템 작동

input_num = int(input()) - 1
grade_subs = np.abs(user_grade_sum - user_grade_sum[input_num + 1]) * 0.1
evaluation_value = user_based_collab[input_num] + grade_subs - user_based_collab_style[input_num]
top10 = evaluation_value.sort_values()[:10].index.tolist()

print(f"<나의 정보>")
print()
print(f"{checkStyle(user_style.iloc[input_num])}")
print(f"데이터분석 : {user_grade.iloc[input_num]['데이터분석']}, 머신러닝 : {user_grade.iloc[input_num]['머신러닝']}, 웹 : {user_grade.iloc[input_num]['웹']}, 디자인 : {user_grade.iloc[input_num]['디자인']}, 총합 : {user_grade_sum[input_num + 1]}")
print(f"강점 : {user_grade.iloc[input_num].sort_values(ascending=False)[:2].index.tolist()}")
print()

print("================================")
for idx, value in enumerate(top10):
    value = value - 1
    print()
    print(f"<추천 {idx + 1}순위 : {value}번째 학생>")
    print()
    print(f"{checkStyle(user_style.iloc[value])}")
    print(f"데이터분석 : {user_grade.iloc[value]['데이터분석']}, 머신러닝 : {user_grade.iloc[value]['머신러닝']}, 웹 : {user_grade.iloc[value]['웹']}, 디자인 : {user_grade.iloc[value]['디자인']}, 총합 : {user_grade_sum[value + 1]}")
    print(f"강점 : {user_grade.iloc[value].sort_values(ascending=False)[:2].index.tolist()}")
    print()
    print("=========================")

<나의 정보>

['속도', '결과', '창업', '밤']
데이터분석 : 4, 머신러닝 : 9, 웹 : 9, 디자인 : 5, 총합 : 27
강점 : ['웹', '머신러닝']


<추천 1순위 : 54번째 학생>

['디테일', '결과', '취업', '밤']
데이터분석 : 10, 머신러닝 : 8, 웹 : 4, 디자인 : 5, 총합 : 27
강점 : ['데이터분석', '머신러닝']


<추천 2순위 : 138번째 학생>

['디테일', '결과', '창업', '밤']
데이터분석 : 9, 머신러닝 : 6, 웹 : 5, 디자인 : 8, 총합 : 28
강점 : ['데이터분석', '디자인']


<추천 3순위 : 4번째 학생>

['속도', '결과', '창업', '밤']
데이터분석 : 4, 머신러닝 : 9, 웹 : 9, 디자인 : 5, 총합 : 27
강점 : ['웹', '머신러닝']


<추천 4순위 : 129번째 학생>

['디테일', '결과', '취업', '밤']
데이터분석 : 10, 머신러닝 : 10, 웹 : 3, 디자인 : 4, 총합 : 27
강점 : ['머신러닝', '데이터분석']


<추천 5순위 : 83번째 학생>

['디테일', '과정', '취업', '아침']
데이터분석 : 6, 머신러닝 : 9, 웹 : 2, 디자인 : 10, 총합 : 27
강점 : ['디자인', '머신러닝']


<추천 6순위 : 59번째 학생>

['속도', '결과', '취업', '밤']
데이터분석 : 8, 머신러닝 : 8, 웹 : 6, 디자인 : 5, 총합 : 27
강점 : ['머신러닝', '데이터분석']


<추천 7순위 : 151번째 학생>

['디테일', '과정', '취업', '밤']
데이터분석 : 10, 머신러닝 : 9, 웹 : 2, 디자인 : 6, 총합 : 27
강점 : ['데이터분석', '머신러닝']


<추천 8순위 : 58번째 학생>

['디테일', '결과', '창업', '밤']
데이터분석 : 6, 머신러닝 : 1, 웹 : 9, 디자인 : 9, 총합 : 25
강점 : ['